# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"> **Air Quality** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## 🗒️ This notebook is divided into the following sections:

1. Download model and batch inference data
2. Make predictions, generate PNG for forecast
3. Store predictions in a monitoring feature group adn generate PNG for hindcast

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Read the API keys and configuration variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

In [ ]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
from mlfs.airquality import util
import os

In [ ]:
# Reload the util module to pick up any changes
import importlib
importlib.reload(util)

In [ ]:
today = datetime.datetime.now() - datetime.timedelta(0)
tomorrow = today + datetime.timedelta(days = 1)
today

## <span style="color:#ff5f27;"> 📡 Connect to Hopsworks Feature Store </span>

In [ ]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 

secrets = hopsworks.get_secrets_api()
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)
country=location['country']
city=location['city']
street=location['street']

## <span style="color:#ff5f27;">🪝 Download the model from Model Registry</span>

### Model Version Information:
- **Version 1**: Baseline (4 weather features only)
- **Version 2**: Enhanced (7 features: weather + 3 lagged PM2.5 values)

We'll load the best performing model from the training pipeline.

In [ ]:
mr = project.get_model_registry()

# Get the latest/best model - change version based on which model won in Notebook 3
retrieved_model = mr.get_model(
    name="air_quality_xgboost_model",
    version=3,  # Use version 2 (enhanced model with lagged features)
)

fv = retrieved_model.get_feature_view()

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

print(f"Loaded model version {retrieved_model.version}")

In [ ]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

## <span style="color:#ff5f27;">✨ Get Weather Forecast Features with Feature View   </span>



In [ ]:
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
# Filter weather data by city to only get forecast for this specific location
batch_data = weather_fg.filter(
    (weather_fg.date >= today) &
    (weather_fg.city == city)
).read()
batch_data

In [ ]:
# Get air quality feature group (version 2 with lagged features)
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=2,
)

## <span style="color:#ff5f27;">🔄 Calculate Lagged Features for Prediction</span>

For the enhanced model (v2), we need the last 3 days of PM2.5 values as features.

In [ ]:
# Fetch the last 3 days of PM2.5 data to calculate lagged features
three_days_ago = today - datetime.timedelta(days=3)
historical_aq_df = air_quality_fg.filter(
    (air_quality_fg.date >= three_days_ago) & 
    (air_quality_fg.city == city) & 
    (air_quality_fg.street == street)
).read()

# Sort by date to ensure proper ordering
historical_aq_df = historical_aq_df.sort_values(by='date')

# Get the last 3 PM2.5 values for lagged features
if len(historical_aq_df) >= 3:
    pm25_values = historical_aq_df['pm25'].tail(3).values
    
    # Add lagged features to batch_data (same value for all forecast days)
    # Explicitly convert to float to avoid dtype issues
    batch_data['pm_25_1_day_lag'] = float(pm25_values[-1])  # Yesterday's PM2.5
    batch_data['pm_25_2_day_lag'] = float(pm25_values[-2])  # 2 days ago
    batch_data['pm_25_3_day_lag'] = float(pm25_values[-3])  # 3 days ago
    
    print(f"Lagged features added:")
    print(f"   1-day lag: {pm25_values[-1]:.2f}")
    print(f"   2-day lag: {pm25_values[-2]:.2f}")
    print(f"   3-day lag: {pm25_values[-3]:.2f}")
else:
    print(f"Warning: Not enough historical data for lagged features")
    print(f"   Found {len(historical_aq_df)} days, need 3")
    # Set to NaN instead of None for numeric columns
    batch_data['pm_25_1_day_lag'] = float('nan')
    batch_data['pm_25_2_day_lag'] = float('nan')
    batch_data['pm_25_3_day_lag'] = float('nan')

batch_data

### <span style="color:#ff5f27;">🤖 Making the predictions</span>

Using the enhanced model with weather features + lagged PM2.5 values.

In [ ]:
# Define feature columns based on model version
# IMPORTANT: Order must match training order!
# Model was trained with lagged features FIRST, then weather features

feature_columns = []

# Add lagged features first if they exist in batch_data
if 'pm_25_1_day_lag' in batch_data.columns:
    feature_columns.extend(['pm_25_1_day_lag', 'pm_25_2_day_lag', 'pm_25_3_day_lag'])
    print(f"Added lagged features")

# Then add weather features
feature_columns.extend([
    'temperature_2m_mean', 
    'precipitation_sum', 
    'wind_speed_10m_max', 
    'wind_direction_10m_dominant'
])

print(f"Using {len(feature_columns)} features: {feature_columns}")

# Make predictions
batch_data['predicted_pm25'] = retrieved_xgboost_model.predict(batch_data[feature_columns])

print(f"\nPredictions generated for {len(batch_data)} days")
batch_data

In [ ]:
batch_data.info()

### <span style="color:#ff5f27;">🤖 Saving the predictions (for monitoring) to a Feature Group</span>

In [ ]:
batch_data['street'] = street
batch_data['city'] = city
batch_data['country'] = country
# Fill in the number of days before the date on which you made the forecast (base_date)
batch_data['days_before_forecast_day'] = range(1, len(batch_data)+1)
batch_data = batch_data.sort_values(by=['date'])
batch_data

In [ ]:
batch_data.info()

### Create Forecast Graph
Draw a graph of the predictions with dates as a PNG and save it to the github repo
Show it on github pages

In [ ]:

pred_file_path = f"{root_dir}/docs/air-quality/assets/img/pm25_forecast.png"
plt = util.plot_air_quality_forecast(city, street, batch_data, pred_file_path)

plt.show()

In [ ]:
# Get or create feature group
monitor_fg = fs.get_or_create_feature_group(
    name='aq_predictions',
    description='Air Quality prediction monitoring',
    version=2,
    primary_key=['city','street','date','days_before_forecast_day'],
    event_time="date"
)

In [ ]:
monitor_fg.insert(batch_data, wait=True)

In [ ]:
# We will create a hindcast chart for only the forecasts made 1 day beforehand
# Filter by city and street to get predictions for this specific sensor only
monitoring_df = monitor_fg.filter(
    (monitor_fg.days_before_forecast_day == 1) &
    (monitor_fg.city == city) &
    (monitor_fg.street == street)
).read()
monitoring_df

In [ ]:
# Use version 2 for actual outcomes (matches training data)
# Filter by city and street to get outcomes for this specific sensor only
air_quality_fg_outcomes = fs.get_feature_group(name='air_quality', version=2)
air_quality_df = air_quality_fg_outcomes.filter(
    (air_quality_fg_outcomes.city == city) &
    (air_quality_fg_outcomes.street == street)
).read()
air_quality_df

In [ ]:
outcome_df = air_quality_df[['date', 'pm25']]
preds_df =  monitoring_df[['date', 'predicted_pm25']]

hindcast_df = pd.merge(preds_df, outcome_df, on="date")
hindcast_df = hindcast_df.sort_values(by=['date'])

# If there are no outcomes for predictions yet, generate some predictions/outcomes from existing data
if len(hindcast_df) == 0:
    hindcast_df = util.backfill_predictions_for_monitoring(weather_fg, air_quality_df, monitor_fg, retrieved_xgboost_model)
hindcast_df

### Plot the Hindcast comparing predicted with forecasted values (1-day prior forecast)

__This graph will be empty to begin with - this is normal.__

After a few days of predictions and observations, you will get data points in this graph.

In [ ]:
# Reload util to get the fix for empty dataframes
importlib.reload(util)

hindcast_file_path = f"{root_dir}/docs/air-quality/assets/img/pm25_hindcast_1day.png"
plt = util.plot_air_quality_forecast(city, street, hindcast_df, hindcast_file_path, hindcast=True)
plt.show()

### Upload the prediction and hindcast dashboards (png files) to Hopsworks


In [ ]:
dataset_api = project.get_dataset_api()
str_today = today.strftime("%Y-%m-%d")
if dataset_api.exists("Resources/airquality") == False:
    dataset_api.mkdir("Resources/airquality")
dataset_api.upload(pred_file_path, f"Resources/airquality/{city}_{street}_{str_today}", overwrite=True)
dataset_api.upload(hindcast_file_path, f"Resources/airquality/{city}_{street}_{str_today}", overwrite=True)

proj_url = project.get_url()
print(f"See images in Hopsworks here: {proj_url}/settings/fb/path/Resources/airquality")

---